In [54]:
Ruedas = {
    "base": {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ", 
         "notch": "", 
         "turnover": ""
    },
    "I"  : {
         "secuencia": "EKMFLGDQVZNTOWYHXUSPAIBRCJ", 
         "notch": "Q", 
         "turnover": "R"
    },
    "II" : {
         "secuencia": "AJDKSIRUXBLHWTMCQGZNPYFVOE", 
         "notch": "E", 
         "turnover": "F"
    },
    "III": {
         "secuencia": "BDFHJLCPRTXVZNYEIWGAKMUSQO", 
         "notch": "V", 
         "turnover": "W"
    },
    "IV" : {
         "secuencia": "ESOVPZJAYQUIRHXLNFTGKDCMWB", 
         "notch": "J", 
         "turnover": "K"
    },
    "V"  : {
         "secuencia": "VZBRGITYUPSDNHLXAWMJQOFECK", 
         "notch": "Z", 
         "turnover": "A"
    },
    "VI" : {
         "secuencia": "JPGVOUMFYQBENHZRDKASXLICTW", 
         "notch": "ZM", 
         "turnover": "AN"
    },
    "VII": {
         "secuencia": "NZJHGRCXMYSWBOUFAIVLPEKQDT", 
         "notch": "ZM", 
         "turnover": "AN"
    },
    "VIII": {
         "secuencia": "FKQHTLXOCBJSPDZRAMEWNIUYGV", 
         "notch": "ZM", 
         "turnover": "AN"
    }
}

Ruedas_griegas = {
    "base" : {
         "secuencia": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "notch": "",
         "turnover": ""
     },
    "beta" : {
         "secuencia": "LEYJVCNIXWPBQMDRTAKZGFUHOS",
         "notch": "",
         "turnover": ""
     },
    "gamma": {
         "secuencia": "FSOKANUERHMBTIYCWLQPZXVGJD",
         "notch": "",
         "turnover": ""
     }

}

Reflectores = {
    "base": "ABCDEFGHIJKLMNOPQRSTUVWXYZ",
    "A"   : "EJMZALYXVBWFCRQUONTSPIKHGD",
    "B"   : "YRUHQSLDPXNGOKMIEBFZCWVJAT",
    "C"   : "FVPJIAOYEDRZXWGCTKUQSBNMHL"
}

In [42]:
class Rotor:
    def __init__(self, rueda, ringstellung: str):      
        """
        Rueda es el mapa de la parte inicial del fichero. 
        """

        # TODO: cambiar self.wiring por self.secuencia para ser consistentes
        print(rueda)
        self.base = Ruedas["base"]["secuencia"]
        self.wiring = rueda["secuencia"]
        self.notch = rueda["notch"]
        self.turnover = rueda["turnover"]
        self.needs_rotation = False
        indice_ring = Ruedas["base"]["secuencia"].find(ringstellung)

        # Aplicamos el ringstellung, moviendo cada letra ciertas posiciones con respecto a la base
        self.wiring = self.wiring[indice_ring:] + self.wiring[:indice_ring]


    def siguiente(self, indice: int):
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = self.base.index(letra_a_buscar)
        return indice_siguiente

    def revertir(self, indice):
        letra_en_base = self.base[indice]
        indice_de_salida = self.wiring.index(letra_en_base)    
        return indice_de_salida
    
    def rotar(self):
        self.base = self.base[1:] + self.base[:1]
        self.wiring = self.wiring[1:] + self.wiring[:1]
        
        if self.base[0] in self.notch:
            # Indicamos así que el rotor de al lado necesita rotar
            self.needs_rotation = True
    
    def ajusta_grundstellung(self, indice):
        self.base = self.base[indice:] + self.base[:indice]
        self.wiring = self.wiring[indice:] + self.wiring[:indice]
        



In [53]:
class Reflector:
    def __init__(self, reflector: str):
        self.wiring = reflector
    
    def reflejar(self, indice):
        """
        `Indice` representa la letra de base[indice].
        """
        letra_a_buscar = self.wiring[indice]
        indice_siguiente = Reflectores["base"].find(letra_a_buscar)
        return indice_siguiente

In [50]:
class Plugboard:
    def __init__(self, parametros):
        """
        El parámetro `mapping` es una lista de duplas, de la forma
        ```
        [(letra1, letra2), (letra3, letra4)]
        ```
        
        De forma que se construirá un mapa tal que letra1 -> letra2, etc...
        """
        
        self.base = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        self.mapping = {}

        # Nos guardamos primero un mapa de la forma A -> A, B -> B, ...
        for letra in self.base:
            self.mapping[letra] = letra

        # Y ahora lo sustituimos por lo que nos han mandado
        for dupla in parametros:
            self.mapping[dupla[0]] = dupla[1]
            self.mapping[dupla[1]] = dupla[0]
    
    def siguiente(self, letra: str):
        return self.mapping[letra]

In [55]:
class Enigma:
    def __init__(self, ruedas, ringstellungs: str, grundstellungs: str, reflector: str, steckern: list[(str, str)]):
        
        if len(ruedas) != len(ringstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del ringstellung")
        if len(ruedas) != len(grundstellungs):
            print("Me has dado un número de ruedas distinto de la configuración del grundstellung")
        
        self.rotores = []
        for rotor, ring in zip(ruedas, ringstellungs):
            self.rotores.append(Rotor(rotor, ring))
        
        # Encontramos los índices correspondientes al grundstellung
        indices = []
        for letra in grundstellungs:
            indices.append(Ruedas["base"]["secuencia"].find(letra))

        for rotor, indice in zip(self.rotores, indices):
            rotor.ajusta_grundstellung(indice)

        self.plugboard = Plugboard(steckern)
        self.reflector = Reflector(reflector)
    
    def cifrar_caracter(self, c: str):
        # Letra -> plugbooard -> rotores -> reflector -> rotores -> plugboard -> salida
        letra = self.plugboard.siguiente(c)

        self.rotores[0].rotar()

        if(not self.rotores[0].needs_rotation and self.rotores[1].needs_rotation):
            self.rotores[1].needs_rotation = False
            self.rotores[1].rotar()
            self.rotores[2].rotar()

        if(self.rotores[0].needs_rotation and not self.rotores[1].needs_rotation):
            self.rotores[0].needs_rotation = False
            self.rotores[1].rotar()

        if(self.rotores[0].needs_rotation and self.rotores[1].needs_rotation):
            self.rotores[0].needs_rotation = False
            self.rotores[1].needs_rotation = False
            self.rotores[1].rotar()
            self.rotores[2].rotar()
        
        # A partir de aquí, necesitamos los índices, que nos serán más cómodos. Retomaremos la letra más tarde
        indice_letra = Ruedas["base"]["secuencia"].find(letra)

        for rotor in self.rotores:
            indice_letra = rotor.siguiente(indice_letra)
        
        indice_letra = self.reflector.reflejar(indice_letra)
    
        for rotor in reversed(self.rotores):
            indice_letra = rotor.revertir(indice_letra)
        
        letra = Ruedas["base"]["secuencia"][indice_letra]

        letra = self.plugboard.siguiente(letra)
        
        return letra

    def cifrar(self, mensaje: str):
        # FIXME cuidado con la configuración inicial
        
        salida = ""

        for letra in mensaje:
            if letra == " ":
                salida = salida + " "
            else:
                salida = salida + self.cifrar_caracter(letra)
        
        return salida


In [62]:
rotores = [Ruedas["VIII"], Ruedas["VI"], Ruedas["V"], Ruedas_griegas["beta"]]
ringstellungs = "LEPE"
grundstellungs = "MEAN"
clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje = "Q"
# mensaje = "QEOB"
# mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"
# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
# mensaje = mensaje[10:len(mensaje)-10]
print(mensaje)

mensaje_cifrado = enigma.cifrar(mensaje)
print(mensaje_cifrado)
# es_conmutativo(mensaje, nueva_enigma.cifrar(mensaje_cifrado)), mensaje_cifrado

{'secuencia': 'FKQHTLXOCBJSPDZRAMEWNIUYGV', 'notch': 'ZM', 'turnover': 'AN'}
{'secuencia': 'JPGVOUMFYQBENHZRDKASXLICTW', 'notch': 'ZM', 'turnover': 'AN'}
{'secuencia': 'VZBRGITYUPSDNHLXAWMJQOFECK', 'notch': 'Z', 'turnover': 'A'}
{'secuencia': 'LEYJVCNIXWPBQMDRTAKZGFUHOS', 'notch': '', 'turnover': ''}
Q
V


In [ ]:
rotores = [Ruedas["V"], Ruedas["VI"], Ruedas["VIII"], Ruedas_griegas['beta']]
ringstellungs = "EPEL"  # 5 16 5 12
grundstellungs = "NAEM" # 14 1 5 13

clavijas = [("A", "E"), ("B", "F"), ("C", "M"), ("D", "Q"), ("H", "U"), ("J", "N"), ("L", "X"), ("P", "R"), ("S", "Z"), ("V", "W")]

enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)
print(enigma.cifrar("QEOB"))


enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)
mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"

# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]
print(mensaje)

nueva_enigma = Enigma(rotores, ringstellungs, grundstellungs, Reflectores["C"], clavijas)

mensaje_cifrado = enigma.cifrar(mensaje)

es_conmutativo(mensaje, nueva_enigma.cifrar(mensaje_cifrado)), mensaje_cifrado

{'secuencia': 'VZBRGITYUPSDNHLXAWMJQOFECK', 'notch': 'Z', 'turnover': 'A'}
{'secuencia': 'JPGVOUMFYQBENHZRDKASXLICTW', 'notch': 'ZM', 'turnover': 'AN'}
{'secuencia': 'FKQHTLXOCBJSPDZRAMEWNIUYGV', 'notch': 'ZM', 'turnover': 'AN'}
{'secuencia': 'LEYJVCNIXWPBQMDRTAKZGFUHOS', 'notch': '', 'turnover': ''}


TypeError: string indices must be integers

In [ ]:
print(Ruedas["V"]["secuencia"])

VZBRGITYUPSDNHLXAWMJQOFECK
